In [15]:
import onnxruntime as ort
import numpy as np
import os
from PIL import Image
from sklearn.metrics import accuracy_score, confusion_matrix

In [16]:
TEST_DIR = "skin_cancer/test" 
ORIGINAL_MODEL = "skin_cancer_resnet.onnx"
QUANTIZED_MODEL = "skin_cancer_mobile_quant.onnx"
IMG_SIZE = (224, 224)

In [17]:
session_float = ort.InferenceSession(ORIGINAL_MODEL)
session_quant = ort.InferenceSession(QUANTIZED_MODEL)

In [18]:
input_name = session_float.get_inputs()[0].name
input_name

'input_image'

In [19]:
classes = sorted([d for d in os.listdir(TEST_DIR) if os.path.isdir(os.path.join(TEST_DIR, d))])
class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
print(f"Classes found: {classes}")

Classes found: ['Benign keratosis-like lesions', 'Melanocytic nevi', 'Melanoma']


In [20]:
def preprocess_image(image_path):
    """
    Load image, resize to 224x224, convert to array.

    """
    img = Image.open(image_path).convert('RGB')
    img = img.resize(IMG_SIZE)
    img_data = np.array(img).astype(np.float32)
    
    # Add Batch Dimension: (224, 224, 3) -> (1, 224, 224, 3)
    img_data = np.expand_dims(img_data, axis=0)
    return img_data

In [31]:
y_true = []
y_pred_float = []
y_pred_quant = []
total_files = sum([len(files) for r, d, files in os.walk(TEST_DIR)])
count = 0

In [32]:
import time
original_times = []
quantized_times = []
for class_name in classes:
    class_dir = os.path.join(TEST_DIR, class_name)
    ground_truth_id = class_to_idx[class_name]
    
    for fname in os.listdir(class_dir):
        if not fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue
            
        # Prepare Data
        
        img_path = os.path.join(class_dir, fname)
        input_data = preprocess_image(img_path)
        
        
        # Run Original
        start_time = time.time()
        out_float = session_float.run(None, {input_name: input_data})[0]
        pred_float = np.argmax(out_float)
        end_time = time.time()
        original_times.append(end_time - start_time)
        
        # Run Quantized
        start_time = time.time()
        out_quant = session_quant.run(None, {input_name: input_data})[0]
        pred_quant = np.argmax(out_quant)
        end_time = time.time()
        quantized_times.append(end_time - start_time)
        
        # Store results
        y_true.append(ground_truth_id)
        y_pred_float.append(pred_float)
        y_pred_quant.append(pred_quant)
        
        count += 1
        if count % 10 == 0:
            print(f"Processed {count}/{total_files} images...", end='\r')

In [33]:
print(np.sum(original_times))
print(np.sum(quantized_times))


23.943447828292847
29.422743558883667


In [34]:
print(original_times[0])
print(quantized_times[0])


0.06894898414611816
0.08478689193725586


In [23]:
acc_float = accuracy_score(y_true, y_pred_float)
acc_quant = accuracy_score(y_true, y_pred_quant)

print(f"Original Model Accuracy:  {acc_float * 100:.2f}%")
print(f"Quantized Model Accuracy: {acc_quant * 100:.2f}%")

Original Model Accuracy:  74.55%
Quantized Model Accuracy: 73.33%


In [25]:
drop = acc_float - acc_quant
print(f"Accuracy Drop:            {drop * 100:.2f}%")

Accuracy Drop:            1.21%


In [28]:
# Show specific cases where they disagreed
disagreements = 0
for i in range(len(y_true)):
    if y_pred_float[i] != y_pred_quant[i]:
        disagreements += 1
        print(f"Img {i}: Original said {classes[y_pred_float[i]]}, Quant said {classes[y_pred_quant[i]]}")

print(f"The models disagreed on {disagreements} images out of {total_files}.")

Img 1: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 3: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 16: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 28: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 82: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 99: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 103: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 132: Original said Melanocytic nevi, Quant said Benign keratosis-like lesions
Img 220: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 225: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 237: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 242: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 249: Original said Melanoma, Quant said Benign keratosis-like lesions
Img 259: Original said Melanoma, Quant